<a href="https://colab.research.google.com/github/ElectroAbhinav/HelloWorld/blob/master/Core_ML_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn
%tensorflow_version 2.x

**LINEAR REGRESSION**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
dftrain= pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') #training data
dfeval= pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') #testing data
print(dftrain.describe())
y_train= dftrain.pop('survived') #removes the column of the surviblity from the data sets and stores it into another variable name of "y_train"
y_eval= dfeval.pop('survived')

         survived         age  n_siblings_spouses       parch        fare
count  627.000000  627.000000          627.000000  627.000000  627.000000
mean     0.387560   29.631308            0.545455    0.379585   34.385399
std      0.487582   12.511818            1.151090    0.792999   54.597730
min      0.000000    0.750000            0.000000    0.000000    0.000000
25%      0.000000   23.000000            0.000000    0.000000    7.895800
50%      0.000000   28.000000            0.000000    0.000000   15.045800
75%      1.000000   35.000000            1.000000    0.000000   31.387500
max      1.000000   80.000000            8.000000    5.000000  512.329200


In [ ]:
categorical_columns= ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric_columns= ['age', 'fare']

feature_column= []
for feature_name in categorical_columns:
  vocabulary = dftrain[feature_name].unique() #gets all the unique values from the feature_name
  #print(vocabulary)
  feature_column.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in numeric_columns:
  feature_column.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))
  
#print(feature_column)

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=12, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use


In [ ]:
train_input_fxn= make_input_fn(dftrain, y_train)
eval_output_fxn= make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est= tf.estimator.LinearClassifier(feature_columns=feature_column)
linear_est.train(train_input_fxn) #training the model
result= linear_est.evaluate(eval_output_fxn) #getting the metrics/standards of the model on Main Dataset(s)

clear_output() #clears the console screen
print(result['accuracy']) #prints the accuracy of the model trained

0.75


If epoch=10, the accuracy of the model is 0.76 but when epoch=12 the accuracy of the model dropped to 0.75!  

In [ ]:
result= list (linear_est.predict(eval_output_fxn))
print(dfeval.loc[2])
print(y_eval.loc[2])
print(result[2]['probabilities'][1])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp24dn0s_k/model.ckpt-240
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                            58
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
emb